In [1]:
# Q1. Getting the embeddings model

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [5]:
user_question = "I just discovered the course. Can I still join it?"
embedding = embedding_model.encode(user_question)

In [6]:
embedding[0]

0.078222655

In [7]:
# Prepare the documents

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [9]:
len(documents)

948

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [11]:
filtered_documents = [document for document in documents if document["course"] == "machine-learning-zoomcamp"]

In [12]:
len(filtered_documents)

375

In [13]:
# Q2. Creating the embeddings

In [14]:
embeddings = []

for document in filtered_documents:
    qa_text = f"{document['question']} {document['text']}"
    embeddings.append(embedding_model.encode(qa_text))

In [15]:
import numpy as np

In [16]:
X = np.array(embeddings)

In [17]:
X.shape

(375, 768)

In [18]:
# Q3. Search

In [19]:
v = embedding
scores = X.dot(v)

In [20]:
max(scores)

0.6506573

In [21]:
# Vector search

In [22]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)

In [23]:
# Q4. Hit-rate for our search engine

In [24]:
import pandas as pd

In [25]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [26]:
from tqdm.auto import tqdm

In [27]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
    }

In [28]:
search_engine = VectorSearchEngine(documents=filtered_documents, embeddings=X)

def search_function(q):
    v = embedding_model.encode(q["question"])
    return search_engine.search(v, num_results=5)

In [29]:
evaluate(ground_truth, search_function)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}

In [30]:
# Q5. Indexing with Elasticsearch

In [31]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [39]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

In [40]:
index_name = "question_answer_embeddings"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

for document, embedding in tqdm(zip(filtered_documents, embeddings)):
    doc = {
        "id": document["id"],
        "question_answer_vector": embedding, 
    }
    es_client.index(index=index_name, document=doc)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x738d218fe0b0>: Failed to establish a new connection: [Errno 111] Connection refused))

In [ ]:
query = {
    "field": "question_answer_vector",
    "query_vector": embedding,
    "k": 5,
    "num_candidates": 10000, 
}

In [37]:
res = es_client.search(index=index_name, knn=query, source=["id"])
res["hits"]["hits"]

[{'_index': 'question_answer_embeddings',
  '_id': '0CeZuJABVSxBJ-83maoS',
  '_score': 1.0,
  '_source': {'id': 'c6a22665'}},
 {'_index': 'question_answer_embeddings',
  '_id': 'WieZuJABVSxBJ-83aql2',
  '_score': 0.80612874,
  '_source': {'id': '0227b872'}},
 {'_index': 'question_answer_embeddings',
  '_id': 'ayeZuJABVSxBJ-83bakk',
  '_score': 0.7538284,
  '_source': {'id': '0a278fb2'}},
 {'_index': 'question_answer_embeddings',
  '_id': 'nSeZuJABVSxBJ-83kqrh',
  '_score': 0.7050337,
  '_source': {'id': '0b8739b7'}},
 {'_index': 'question_answer_embeddings',
  '_id': 'YSeZuJABVSxBJ-83i6pd',
  '_score': 0.69782126,
  '_source': {'id': 'f826cba4'}}]